# ZHIPU AI

本 Notebook 展示了如何在 LangChain 中使用 [ZHIPU AI API](https://open.bigmodel.cn/dev/api) 以及 langchain.chat_models.ChatZhipuAI。

>[*GLM-4*](https://open.bigmodel.cn/) 是一个多语言、符合人类意图的大语言模型，具备问答、多轮对话和代码生成等能力。新一代基础模型 GLM-4 的整体性能相比上一代得到显著提升，支持更长的上下文；更强的多模态能力；支持更快的推理速度、更多的并发，大幅降低推理成本；同时，GLM-4 增强了智能体的能力。

## 开始使用
### 安装
首先，请确保您的 Python 环境已安装 zhipuai 包。运行以下命令：

In [ ]:
#!pip install --upgrade httpx httpx-sse PyJWT

### 导入所需模块
安装完成后，请将必要的模块导入您的 Python 脚本：

In [ ]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

### 设置你的 API 密钥
请登录 [ZHIPU AI](https://open.bigmodel.cn/login?redirect=%2Fusercenter%2Fapikeys) 以获取 API 密钥，从而访问我们的模型。

In [ ]:
import os

os.environ["ZHIPUAI_API_KEY"] = "zhipuai_api_key"

### 初始化 ZHIPU AI 聊天模型
以下是如何初始化聊天模型的步骤：

In [ ]:
chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

### 基本用法
像这样使用系统消息和用户消息来调用模型：

In [ ]:
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a poet."),
    HumanMessage(content="Write a short poem about AI in four lines."),
]

In [ ]:
response = chat.invoke(messages)
print(response.content)  # Displays the AI-generated poem

## 高级特性
### 流式支持
为了实现连续交互，请使用流式特性：

In [ ]:
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
streaming_chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [ ]:
streaming_chat(messages)

### 异步调用
对于非阻塞调用，请使用异步方法：

In [ ]:
async_chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [ ]:
response = await async_chat.agenerate([messages])
print(response)

### 结合函数调用使用

GLM-4 模型也可以结合函数调用使用。使用以下代码运行一个简单的 LangChain json_chat_agent。

In [ ]:
os.environ["TAVILY_API_KEY"] = "tavily_api_key"

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_json_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/react-chat-json")
llm = ChatZhipuAI(temperature=0.01, model="glm-4")

agent = create_json_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": "what is LangChain?"})